In [ ]:
#import data from bulk CSV downloads

file_prefix = 'documents_signed_by_'
pres_names = ['joseph_r_biden_jr', 'donald_trump', 'barack_obama', 'george_w_bush', 'william_j_clinton']
file_suffix = '_of_type_presidential_document_and_of_presidential_document_type_executive_order.csv'
colnames = ['executive_order_number', 'title', 'signing_date', 'disposition_notes']

df_all = pd.DataFrame(columns = colnames)
for name in pres_names:
    filename = file_prefix + name + file_suffix
    df = pd.read_csv(filename)
    df_subset = df[colnames]
    print(name)
    df_all = df_all.append(df_subset)
print(df_all.head())
print(df_all.shape)